## WeatherPy

In [67]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from config import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "Output/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [7]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

611

In [81]:
ten = cities[0:10]

In [82]:
# Create a DataFrame for the random cities
city_Weather = pd.DataFrame({"City" : ten})
city_Weather["Lat"] = ""
city_Weather["Lng"] = ""
city_Weather["Max Temps"] = ""
city_Weather["Humidity"] = ""
city_Weather["Cloudiness"] = ""
city_Weather["Wind Speed"] = ""
city_Weather["Country"] = ""
city_Weather["Date"] = ""

city_Weather.head(10)

,City,Lat,Lng,Max Temps,Humidity,Cloudiness,Wind Speed,Country,Date
0,qaanaaq,,,,,,,,
1,cidreira,,,,,,,,
2,guerrero negro,,,,,,,,
3,nizhneyansk,,,,,,,,
4,mahebourg,,,,,,,,
5,innisfail,,,,,,,,
6,taolanaro,,,,,,,,
7,mataura,,,,,,,,
8,provideniya,,,,,,,,
9,port elizabeth,,,,,,,,


## Perform API Calls

In [112]:
# Using the Open Weather API gather information about the weather of the cities
time.sleep(1)
for index, row in city_Weather.iterrows():
    place = row["City"]
    
    query_url = f"{url}appid={weather_api_key}&q={place}&units={units}"
    weather_response = requests.get(query_url)
    weather_json = weather_response.json()
    
    try:
        city_Weather.loc[index, "Lat"] = weather_json["coord"]["lat"]
        city_Weather.loc[index, "Lng"] = weather_json["coord"]["lon"]
        city_Weather.loc[index, "Max Temps"] = weather_json["main"]["temp_max"]
        city_Weather.loc[index, "Humidity"] = weather_json["main"]["humidity"]
        city_Weather.loc[index, "Cloudiness"] = weather_json["clouds"]["all"]
        city_Weather.loc[index, "Wind Speed"] = weather_json["wind"]["speed"]
        city_Weather.loc[index, "Country"] = weather_json["sys"]["country"]
        city_Weather.loc[index, "Date"] = weather_json["dt"]
        print(f"Processing Record of {index} of Set 1 | {place}")
    except:
        print(f"City not found. Skipping ... ")

Processing Record of 0 of Set 1 | qaanaaq
Processing Record of 1 of Set 1 | cidreira
Processing Record of 2 of Set 1 | guerrero negro
City not found. Skipping ... 
Processing Record of 4 of Set 1 | mahebourg
Processing Record of 5 of Set 1 | innisfail
City not found. Skipping ... 
Processing Record of 7 of Set 1 | mataura
Processing Record of 8 of Set 1 | provideniya
Processing Record of 9 of Set 1 | port elizabeth


## Convert Raw Data to DataFrame

In [113]:
# Export the city weather dat to a csv
city_Weather.to_csv(output_data_file)

# View the dataframe
city_Weather.head()

,City,Lat,Lng,Max Temps,Humidity,Cloudiness,Wind Speed,Country,Date
0,qaanaaq,77,-69.3632,14.31,72,0,2.55,GL,1612914220
1,cidreira,-30,-50.2056,74.75,81,3,18.9,BR,1612914256
2,guerrero negro,27,-114.061,63.73,62,29,12.97,MX,1612914250
3,nizhneyansk,,,,,,,,
4,mahebourg,-20,57.7,77,83,40,9.24,MU,1612914227


In [123]:
# city_Weather["Lat"] = city_Weather.to_numeric(city_Weather["Lat"])
# city_Weather["Lat"] = city_Weather["Lat"].astype(float)
city_Weather["Lat"]

TypeError: '>=' not supported between instances of 'int' and 'str'

## Inspect the data and remove the cities where the humidity > 100%